In [1]:
import torch

cuda_available = torch.cuda.is_available()
print("CUDA is available:", cuda_available)


CUDA is available: True


In [6]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
path = "checkpoints/pcba_denoise.pth"
ckpt = torch.load(path, map_location=device)


In [5]:
from utils.models.ScoreNetwork_A import ScoreNetworkA
from utils.models.ScoreNetwork_X import ScoreNetworkX, ScoreNetworkX_GMH


def load_model(params):
    params_ = params.copy()
    model_type = params_.pop('model_type', None)
    if model_type == 'ScoreNetworkX':
        model = ScoreNetworkX(**params_)
    elif model_type == 'ScoreNetworkX_GMH':
        model = ScoreNetworkX_GMH(**params_)
    elif model_type == 'ScoreNetworkA':
        model = ScoreNetworkA(**params_)
    else:
        raise ValueError(f"Model Name <{model_type}> is Unknown")
    return model

def load_model_from_ckpt(params, state_dict, device):
    model = load_model(params)
    if 'module.' in list(state_dict.keys())[0]:
        # strip 'module.' at front; for DataParallel models
        state_dict = {k[7:]: v for k, v in state_dict.items()}
    model.load_state_dict(state_dict)
    if isinstance(device, list):
        if len(device) > 1:
            model = torch.nn.DataParallel(model, device_ids=device)
    else:
        model = model.to(device)
    return model

In [10]:
denoise_x = load_model_from_ckpt(ckpt['params_x'], ckpt['x_state_dict'], device)
denoise_adj = load_model_from_ckpt(ckpt['params_adj'], ckpt['adj_state_dict'], device)

In [12]:
print(denoise_x)
# print(denoise_adj)

ScoreNetworkX(
  (layers): ModuleList(
    (0): DenseGCNConv(118, 32)
    (1): DenseGCNConv(32, 32)
  )
  (final): MLP(
    (linears): ModuleList(
      (0): Linear(in_features=182, out_features=364, bias=True)
      (1): Linear(in_features=364, out_features=364, bias=True)
      (2): Linear(in_features=364, out_features=118, bias=True)
    )
  )
)


In [25]:
from dataset.get_datasets import get_dataset
import os
import pandas as pd
class Args:
    def __init__(self):
        self.dataset = 'ogbg-molbace'
        self.device = device
        self.batch_size = 128
        
args = Args()


labeled_dataset = get_dataset(args, './raw_data')

In [26]:
mol = labeled_dataset[0]
mol.x[:,0].view(-1) + 1

tensor([8, 6, 6, 6, 7, 6, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 6, 7, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6])

In [27]:
smile_path = os.path.join('./raw_data', '_'.join(args.dataset.split('-')), 'mapping/mol.csv.gz')
smiles_df = pd.read_csv(smile_path, compression='gzip', usecols=['smiles'])
smiles = smiles_df['smiles'].tolist() 

In [28]:
print(smiles[0])

O1CC[C@@H](NC(=O)[C@@H](Cc2cc3cc(ccc3nc2N)-c2ccccc2C)C)CC1(C)C
